In [1]:
import torch
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import yfinance as yf
from datetime import datetime
import numpy as np
import seaborn as sns
import wrds
import statsmodels.api as sm

plt.style.use('seaborn-v0_8-notebook')
eps = 1e-8
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['figure.figsize'] = [10, 6]
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['figure.autolayout'] = True

conn = wrds.Connection()
start = '1965-01-01'
end = '2025-01-01'

Loading library list...
Done


In [2]:
month_end_dates = pd.date_range(
    start=start,
    end=end,
    freq='ME'
)

factors = conn.raw_sql(
    f"""
    SELECT date, mktrf, smb, hml, rf
    FROM ff.factors_daily
    WHERE date BETWEEN '{start}' AND '{end}'
    """,
    date_cols = ['date']
).set_index('date')

factors = (
    factors.replace([np.inf, -np.inf], np.nan)
    .fillna(0)
    .astype(float)
)

factors

,mktrf,smb,hml,rf
date,,,,
1965-01-04,-0.0045,0.0070,-0.0008,0.00014
1965-01-05,0.0049,0.0038,-0.0009,0.00014
1965-01-06,0.0034,0.0018,0.0042,0.00014
1965-01-07,0.0040,0.0008,0.0019,0.00014
1965-01-08,0.0017,0.0017,-0.0020,0.00014
...,...,...,...,...
2024-12-24,0.0111,-0.0009,-0.0005,0.00017
2024-12-26,0.0002,0.0104,-0.0019,0.00017
2024-12-27,-0.0117,-0.0066,0.0056,0.00017


In [3]:
ivol_df = pd.read_csv(
    './ivol_part_3.csv',
    index_col=0
)

ivol_df.index = pd.to_datetime(ivol_df.index)

ivol_df

,56805,42324,91558,25032,26172,75780,13273,87499,79093,20109,...,81263,61867,84815,22543,77646,85205,82655,79255,17976,13712
1965-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1965-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1965-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1965-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1965-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
for i, col in enumerate(ivol_df.columns):
    print(f"Processing column {i} out of {len(ivol_df.columns)}")
    permno = col
    ret_df = (
        conn.raw_sql(
        f"""
        SELECT date, ret
        FROM crsp.dsf
        WHERE date BETWEEN '{start}' AND '{end}'
        AND permno = {permno}
        """,
        date_cols = ['date'])
        .set_index('date')
        .merge(
            factors,
            left_index=True,
            right_index=True,
            how='left'
        )
    )
    ret_df['ex_ret'] = ret_df['ret'] - ret_df['rf']

    for idx in ivol_df.index:
        time_range_mask = (
            (ret_df.index >= idx - pd.DateOffset(months = 1)) * (ret_df.index <= idx)
        )
        X = np.asarray(
            ret_df[['mktrf', 'smb', 'hml']][time_range_mask]
        )
        y = np.asarray(
            ret_df[['ex_ret']][time_range_mask]
        )

        if X.shape[0] != 0:
            X = sm.add_constant(X)
            model = sm.OLS(y, X).fit()
            ivol = np.std(
                model.resid
            )
            ivol_df.loc[idx, col] = ivol

Processing column 0 out of 8425
Processing column 1 out of 8425
Processing column 2 out of 8425
Processing column 3 out of 8425
Processing column 4 out of 8425
Processing column 5 out of 8425
Processing column 6 out of 8425
Processing column 7 out of 8425
Processing column 8 out of 8425
Processing column 9 out of 8425
Processing column 10 out of 8425
Processing column 11 out of 8425
Processing column 12 out of 8425
Processing column 13 out of 8425
Processing column 14 out of 8425
Processing column 15 out of 8425
Processing column 16 out of 8425
Processing column 17 out of 8425
Processing column 18 out of 8425
Processing column 19 out of 8425
Processing column 20 out of 8425
Processing column 21 out of 8425
Processing column 22 out of 8425
Processing column 23 out of 8425
Processing column 24 out of 8425
Processing column 25 out of 8425
Processing column 26 out of 8425
Processing column 27 out of 8425
Processing column 28 out of 8425
Processing column 29 out of 8425
Processing column 30

In [6]:
ivol_df.to_csv('./ivol_part_3.csv')